In [1]:
from tqdm.notebook import tqdm
import numpy as np
import  pandas as pd
import pymorphy2
import itertools


# ignore fucking warnings
import warnings
warnings.filterwarnings("ignore")


import nltk
from nltk.tokenize import RegexpTokenizer
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/lazarev/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# preprocessing

In [14]:
nltk.download('stopwords')
# pymorphy2 анализатор
morph = pymorphy2.MorphAnalyzer()
# стоп слова из nltk
stops = nltk.corpus.stopwords.words('russian')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/lazarev/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [15]:
word_tokenizer = RegexpTokenizer(r'[a-zа-яёЁА-ЯA-Z]+|[^\w\s]|\d+')
sent_tokenizer = lambda sent: nltk.sent_tokenize(sent, language="russian")

In [16]:
def proccess_text(text):
    text= text.lower()
    sents = sent_tokenizer(text)
    words = list(
        itertools.chain.from_iterable(
            word_tokenizer.tokenize_sents(sents)))
    return [x for x in [morph.normal_forms(word)[0] for word in words ]\
            if x not in stops]

In [17]:
dataframe = pd.read_csv("data/data_ruSentNE.csv")
dataframe.head(10)

,text,entity,entity_tag,entity_pos_start_rel,entity_pos_end_rel,category
0,"Джеймс «Бадди» Макгирт (James (Buddy) McGirt, ...",спортсмена,PROFESSION,86,96,0
1,«За всю нашу долгую карьеру нам довелось играт...,музыкантов,PROFESSION,258,268,0
2,"Ранее, 7 декабря, толпа болельщиков перекрыла ...",Егоре Свиридове,PERSON,175,190,0
3,"В субботу, 21 июля 2018 года, на арене СК «Оли...",россиянина,NATIONALITY,122,132,0
4,Представитель талибов Забиулла Муджахид в твит...,американских захватчика,PROFESSION,109,132,-1
5,"Астронавт Пирс Селлерс, который брал с собой в...",Пирс Селлерс,PERSON,10,22,0
6,Президент США Дональд Трамп охарактеризовал си...,Президент США,PROFESSION,0,13,0
7,"По словам Мнучина, «Мадуро цепляется за Кубу, ...",Мадуро,PERSON,20,26,-1
8,Поводом для возможного задержания якобы стали ...,Макафи,PERSON,72,78,-1
9,Джон Леннон был убит в 1980 году одним из его ...,охранником,PROFESSION,66,76,0


In [18]:
data_text = dataframe['text']
for i in tqdm(range(len(dataframe))):
    # print(data_text.loc[i])
    data_text.iloc[i] = ' '.join(proccess_text(data_text.loc[i]))
    

data_text = dataframe['entity']
for i in tqdm(range(len(dataframe))):
    # print(data_text.loc[i])
    data_text.iloc[i] = ' '.join(proccess_text(data_text.loc[i]))
dataframe.to_csv('data/data_ruSentNE_lemmatized.csv', index=False)

# препроцесинг занимает порядка двух часов (из-за лематизации), поэтому 
# рекомендуется загрузить сразу обработанную версию


  0%|          | 0/9482 [00:00<?, ?it/s]

  0%|          | 0/9482 [00:00<?, ?it/s]

In [19]:
dataframe = pd.read_csv("data/data_ruSentNE_lemmatized.csv")
dataframe.head(10)

,text,entity,entity_tag,entity_pos_start_rel,entity_pos_end_rel,category
0,джеймс « бадди » макгирт ( james ( buddy ) mcg...,спортсмен,PROFESSION,86,96,0
1,« весь наш долгий карьера довестись играть мно...,музыкант,PROFESSION,258,268,0
2,"ранее , 7 декабрь , толпа болельщик перекрыть ...",егор свиридов,PERSON,175,190,0
3,"суббота , 21 июль 2018 год , арена ск « олимпи...",россиянин,NATIONALITY,122,132,0
4,представитель талиб забиулла муджахид твиттер ...,американский захватчик,PROFESSION,109,132,-1
5,"астронавт пирс селлерс , который брать полёт м...",пирс селлерс,PERSON,10,22,0
6,президент сша дональд трамп охарактеризовать с...,президент сша,PROFESSION,0,13,0
7,"слово мнучин , « мадурый цепляться куба , оста...",мадурый,PERSON,20,26,-1
8,повод возможный задержание якобы стать подозре...,макафи,PERSON,72,78,-1
9,джон леннон убить 1980 год фанат — 25 - летний...,охранник,PROFESSION,66,76,0


# undersampling

In [21]:
dataframe = pd.read_csv("data/data_ruSentNE_lemmatized.csv")
dataframe.head(10)

,text,entity,entity_tag,entity_pos_start_rel,entity_pos_end_rel,category
0,джеймс « бадди » макгирт ( james ( buddy ) mcg...,спортсмен,PROFESSION,86,96,0
1,« весь наш долгий карьера довестись играть мно...,музыкант,PROFESSION,258,268,0
2,"ранее , 7 декабрь , толпа болельщик перекрыть ...",егор свиридов,PERSON,175,190,0
3,"суббота , 21 июль 2018 год , арена ск « олимпи...",россиянин,NATIONALITY,122,132,0
4,представитель талиб забиулла муджахид твиттер ...,американский захватчик,PROFESSION,109,132,-1
5,"астронавт пирс селлерс , который брать полёт м...",пирс селлерс,PERSON,10,22,0
6,президент сша дональд трамп охарактеризовать с...,президент сша,PROFESSION,0,13,0
7,"слово мнучин , « мадурый цепляться куба , оста...",мадурый,PERSON,20,26,-1
8,повод возможный задержание якобы стать подозре...,макафи,PERSON,72,78,-1
9,джон леннон убить 1980 год фанат — 25 - летний...,охранник,PROFESSION,66,76,0


In [22]:
(dataframe[dataframe["category"] == 0].shape, 
 dataframe[dataframe["category"] == 1].shape, 
 dataframe[dataframe["category"] == -1].shape)

((6819, 6), (1218, 6), (1445, 6))

In [23]:
random_indexes = np.random.choice(np.arange(dataframe[dataframe["category"] == 0].shape[0]), 
                                  dataframe[dataframe["category"] == 0].shape[0]//6)
random_indexes.shape

(1136,)

In [24]:
dataframe = pd.concat((
    dataframe[dataframe["category"] == 0].iloc[random_indexes],
    dataframe[dataframe["category"] == 1],
    dataframe[dataframe["category"] == -1],
))

In [25]:
dataframe.shape

(3799, 6)

In [26]:
dataframe.to_csv('data/data_ruSentNE_lemmatized.csv', index=False)

# Final data

In [57]:
dataframe = pd.read_csv("../data/final_data.csv", delimiter='\t')
dataframe.head()

,sentence,entity,entity_tag,entity_pos_start_rel,entity_pos_end_rel
0,Абдул Реза Шайхулислами выразил готовность Ира...,Абдул Реза Шайхулислами,PERSON,0,23
1,Абдул Реза Шайхулислами выразил готовность Ира...,афганцев,NATIONALITY,157,165
2,Абдул Реза Шайхулислами выразил готовность Ира...,правительству Афганистана,ORGANIZATION,68,93
3,В своем выступлении Абдул Реза Шайхулислами от...,Абдул Реза Шайхулислами,PERSON,20,43
4,"В свою очередь, Зорар Ахмад Мокбел заявил о то...",Ирана,COUNTRY,86,91


In [58]:
dataframe['category'] = 1

In [59]:
dataframe.to_csv('../data/data_ruSentNE_lemmatized.csv', index=False)

In [5]:
df_1 = pd.read_csv(f"data/laptop_trainval.csv")
df_2 = pd.read_csv(f"data/restaraunts_trainval.csv")
df = pd.concat([df_1, df_2])
df_1_test = pd.read_csv(f"data/laptop_test.csv")
df_2_rest = pd.read_csv(f"data/restaraunts_test.csv")
df_test = pd.concat([df_1_test, df_2_rest])

In [ ]:
df